In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score
import re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib

In [4]:
df = pd.read_csv('datasets_483_982_spam.csv',encoding='latin-1')

In [5]:
df.head()

v1                                                 v2 Unnamed: 2  \
0   ham  Go until jurong point, crazy.. Available only ...        NaN   
1   ham                      Ok lar... Joking wif u oni...        NaN   
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3   ham  U dun say so early hor... U c already then say...        NaN   
4   ham  Nah I don't think he goes to usf, he lives aro...        NaN   

  Unnamed: 3 Unnamed: 4  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [6]:
new_data = df[['v1','v2']]

In [7]:
new_data.head()

v1                                                 v2
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...

In [8]:
# 1 means not spam
# 0 means spam
new_data['v1'] = new_data['v1'].apply(lambda x: 1 if x == 'ham' else 0)

C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
new_data.head()

v1                                                 v2
0   1  Go until jurong point, crazy.. Available only ...
1   1                      Ok lar... Joking wif u oni...
2   0  Free entry in 2 a wkly comp to win FA Cup fina...
3   1  U dun say so early hor... U c already then say...
4   1  Nah I don't think he goes to usf, he lives aro...

In [10]:
stop_words = set(stopwords.words('english'))
def nlp_preprocessing(total_text, index, column):
    if type(total_text) is not int:
        string = ""
        total_text = re.sub('[^a-zA-Z0-9\n]', ' ', total_text)
        total_text = re.sub('\s+',' ', total_text)
        total_text = total_text.lower()
        for word in total_text.split():
            if not word in stop_words:
                string += word + " "
        new_data[column][index] = string

In [11]:
for index, row in new_data.iterrows():
    if type(row['v2']) is str:
        nlp_preprocessing(row['v2'], index, 'v2')
    else:
        print("there is no text description for id:",index)

C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Acer\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
new_data.head()

v1                                                 v2
0   1  go jurong point crazy available bugis n great ...
1   1                           ok lar joking wif u oni 
2   0  free entry 2 wkly comp win fa cup final tkts 2...
3   1               u dun say early hor u c already say 
4   1            nah think goes usf lives around though

In [13]:
text = new_data['v2'][0]
print(text) #using this text when we canvert into a pickle file

go jurong point crazy available bugis n great world la e buffet cine got amore wat 


In [14]:
y = new_data['v1']
X = new_data['v2']

In [15]:
count_vect = CountVectorizer()
count_vect.fit(X)
joblib.dump(count_vect, 'count_vect.pkl')
X1 = count_vect.transform(X)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.33, random_state=42)

In [17]:
clf = SGDClassifier(alpha=0.0001, penalty='l2', loss='log', random_state=42)
clf.fit(X_train, y_train)
predict_y = clf.predict(X_test)
acc = accuracy_score(y_test, predict_y)

C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [18]:
print(f"For alpha value 0.0001 i got accuracy is {acc*100}")

For alpha value 0.0001 i got accuracy is 98.15116911364872


In [19]:
joblib.dump(clf, 'model.pkl')

C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\Acer\Anaconda3\lib\site-packages\sklearn\externals\joblib\numpy_pickle.py:93: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


['model.pkl']

In [20]:
clf = joblib.load('model.pkl')
count_vect = joblib.load('count_vect.pkl')

In [21]:
text='Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [22]:
pred = clf.predict(count_vect.transform([text]))
if pred[0] ==1:
    prediction = "Not spam"
else:
    prediction = "spam"


In [23]:
prediction

'Not spam'